# Improve the sustainable management of water networks: develop a water demand forecasting algorithm with neural networks in Python

![alt text](MLP.png "Title")

What could we do with the knowledge of our water network future consumption? Surely, we can consistently improve and optimize the way we use water. For instance, knowing the consumption of the following day would permit an optimal selection of the pump operation, with consequent important savings in terms of energy and money. This is one of the reasons why the development of reliable water demand forecasting algorithms is a really hot topic in the scientific community. Clearly, water demand forecasting is not an easy task due the strong stochastic behavior of our consumption, which depend mainly by our habits. However, in the latest decades, we have a new ally for the development of such algorithm: the increase of data availability. Therefore, during this workshop we will see how we can develop a powerful algorithm for predicting the future water demand of a water network, based on the data-driven idea that is at the basis of the machine and deep learning philosophy. Therefore, we will take some data and create and ANN algorithm step-by-step using Python, analyzing the results and the methodology. Hence the workshop will be developed as follows:
- Introduction to the importance of sustainability of water networks, and on the new challenge of today.
- Discuss the idea of water demand forecasting and give a closer look to the data-driven idea behind machine learning algorithms.
- Develop a step-by-step code that, starting form some data analysis, will arrive at the development of a neural network model for predicting water demand time series.
- Analyze the output, and discuss the potential of such approach.


### Time to import some foundamental libraries of Python

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from keras.models import Sequential
from keras.layers import Dense
import datetime
from sklearn.preprocessing import MinMaxScaler


### Now lets import our data and take a look at them

In [ ]:
df = pd.read_csv('demand_ts.csv', sep=';')
df.index = pd.to_datetime(df['index'],format="%Y-%m-%d %H:%M:%S")
df.drop(columns=df.columns[[0]], inplace=True)

In [ ]:
print(df)

## Always important to see graphically what is inside our data
The below code will allow us to see the whole time series content


In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_minor_locator(mdates.MonthLocator(interval=6))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m')) 
ax.format_xdata = mdates.DateFormatter('%Y-%m')
ax.plot(df, linewidth=1.5, label = 'Hourly data')   
ax.plot(df.resample('D').mean(), linewidth=1.5, label = 'Daily data') 
ax.set_ylabel('(L/s)', fontsize=22)
ax.legend()
ax.grid(True)       
fig.autofmt_xdate()

In [ ]:
start_date = datetime.datetime.strptime('2013-04-01', '%Y-%m-%d')
end_date = start_date + datetime.timedelta(days = 2)
fig, ax = plt.subplots(figsize=(15,6))
ax.xaxis.set_major_locator(mdates.HourLocator(byhour=0))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d')) 
ax.format_xdata = mdates.DateFormatter('%y-%m-%d')
ax.plot(df[start_date:end_date], linewidth=1.5)
ax.set_ylabel('(L/s)', fontsize=22)
ax.grid(True)          
fig.autofmt_xdate()

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
fig, ax = plt.subplots(figsize=(15,6))
plot_pacf(df, lags=200, method='ywm', ax=ax)
ax.set_xlabel('lag', fontsize=22)
ax.set_title('')
ax.grid(True)

## Past observation are our best ally
This analysis allows us to understand the autocorrelations that existis inside our data. Yes, past observation are our best ally during the prediction task. Therefore, our next step is to manipulate the data carefully in order to prepare them for the neural network model. How? We will use some coding skills


In [ ]:
def lag_selector(dataset, lista_lag):
    dataset_copy = dataset.copy()
    n_lags = len(lista_lag)
    n = dataset_copy.index.size
    for i in range(n_lags):
        if i == 0:
            lag = lista_lag[i]
            df_lag = dataset_copy.copy()
            for j in range(lag):
                dataset = dataset.drop(dataset.index[[0]])
                df_lag = df_lag.drop(df_lag.index[[n-j-1]])  
            counter = 0
            for col in dataset_copy.columns:
                name = col + '_lag'+str(lag)
                dataset[name] = df_lag.values[:, counter]
                counter = counter + 1
        else:
            lag = lista_lag[i]
            lag0 = lista_lag[0]
            df_lag = dataset_copy.copy()
            for j in range(lag0-lag):
                df_lag = df_lag.drop(df_lag.index[[0]])
            for j in range(lag):
                a = df_lag.index.size
                df_lag = df_lag.drop(df_lag.index[[a-1]])  
            counter = 0
            for col in dataset_copy.columns:
                name = col + '_lag'+str(lag)
                dataset[name] = df_lag.values[:, counter]
                counter = counter + 1
    n_var = dataset.columns.size
    return dataset


dataset_multi = lag_selector(dataset = df,
                            lista_lag = [24, 23, 3, 2, 1])

The method above is made to compose the dataset in columns, where each columns is the same type of variable, but referred to a different time step. For instance, the value in the column lag3 refers to 3 time step (in this case 3 hours) before the original column

In [ ]:
print(dataset_multi)

### We need also scaler
Neural netwroks wants data in a precise manner. Shortly, it is better to provide the data in a range between 0 and 1. What is usually dane is to scale the dataset with a min max scaler. Take a look below.  


In [ ]:
scaler =  MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(dataset_multi.values)
# need this for later
scaler_univ =  MinMaxScaler(feature_range=(0, 1))
scaled_univ = scaler_univ.fit_transform(df.values)

# take a look
print(scaled)

Now we have to create a function to carefully prepare and divide our data into a sample for the training of the model, and a sample for testing the model. Why? It is a well accepted way to build a data-driven model. We can use a portion of our model to tune the hyperparameter of our data-driven, in this case the ANN. The remainig data are needed to see if what we did was correct, hence if our model was fitted correctly.

In [ ]:
def test_train_splitter(scaled_dataset, split_point):
    
    seq_to_seq_dataset = scaled_dataset[range(0, scaled_dataset.shape[0], 1), :]
    # i look only to the dimension of the first col
    n_train_hours = int(round(scaled_dataset[:,0].size*split_point)) 
    # divide
    train = seq_to_seq_dataset[:n_train_hours, :]
    test = seq_to_seq_dataset[n_train_hours:, :]
# =========================================================================
    train_X, train_y = train[:, 1:], train[:, 0:1]            
    test_X, test_y = test[:, 1:], test[:, 0:1]
    #reshape input . 
    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    train_y = train_y.reshape((train_y.shape[0], 1, train_y.shape[1]))
    train_y = train_y[:,:,::-1]
    test_y = test_y.reshape((test_y.shape[0], 1, test_y.shape[1]))
    test_y = test_y[:,:,::-1]
                
    return train_X, train_y, test_X, test_y

train_X, train_y, test_X, test_y = test_train_splitter(scaled_dataset = scaled, split_point=0.9)


In [ ]:
fig, ax = plt.subplots(figsize=(15,6))

ax.plot(range(train_y[:,0,0].size) , train_y[:,0,0], linewidth=1.5, label = 'training')   
ax.plot(range(train_y[:,0,0].size, scaled[:,0].size), test_y[:,0,0], linewidth=1.5, label = 'testing') 
#ax.set_ylabel('(L/s)', fontsize=22)
ax.grid(True)       
ax.legend(fontsize=18)
fig.autofmt_xdate()

## Create the model
This is a very important part. Normally, we should pay particular attention to the creation of the model, and we should choose wisely the typology of the model (we could even use a different neural network, like a recurrent one) and also on the hyperparameter selection. Which optimizes shall we use? how many layers? how many neurons? These question can be solved with experince and with search methods like a grid search. In other words, usually we should test a lot of model configuration to find the best one. For this time, we use a not optimised deep neural network, knowing that we could improve our work!

In [ ]:
model = Sequential()      
model.add(Dense(96, input_shape=(train_X.shape[1],train_X.shape[2]),
                     activation='relu'))  
model.add(Dense(96, activation='relu'))
model.add(Dense(1))
model.summary()

## Fit our model
Now we have to fit our model on our data. This means that the model is trained over our training sample and its weights are adjusted in order to make the prediction task.

In [ ]:
model.compile(loss = 'mse', 
              optimizer = 'Adam', 
              metrics = ['accuracy'])              
hist = model.fit(train_X, train_y, epochs=20, 
                  batch_size=24, validation_data=(test_X, test_y), verbose=2, shuffle=False)

## Let's take a look at the training

In [ ]:
plt.figure(figsize=(14, 4))
plt.plot(hist.history['loss'], label='train')
plt.plot(hist.history['val_loss'], label='test')
plt.legend()
plt.show()

## Now we can use the model to make the preditictions

In [ ]:
trainPredict = model.predict(train_X)
trainPredict = np.fliplr(trainPredict)
testPredict = model.predict(test_X)
testPredict = np.fliplr(testPredict)


## Remember that we scaled everything. We need to revert.

In [ ]:
# reshape into original
train_y = train_y.reshape((train_y.shape[0], 1))
test_y = test_y.reshape((test_y.shape[0], 1))
train_y = scaler_univ.inverse_transform(np.fliplr(train_y))
test_y = scaler_univ.inverse_transform(np.fliplr(test_y))
real_concat = np.concatenate((train_y, test_y), axis=None) 
# for predictions
trainPredict = trainPredict.reshape((trainPredict.shape[0], 1))
testPredict = testPredict.reshape((testPredict.shape[0], 1))
trainPredict = scaler_univ.inverse_transform(trainPredict)
testPredict = scaler_univ.inverse_transform(testPredict)
model_concat = np.concatenate((trainPredict, testPredict), axis=None)

In [ ]:
df_real = pd.DataFrame(index=range(dataset_multi.index.size), 
                            columns=['h average'])
df_forcasted = pd.DataFrame(index=range(dataset_multi.index.size), 
                                 columns=['h average'])
df_train = pd.DataFrame(index=range(dataset_multi.index.size),
                             columns=['h average'])
df_test = pd.DataFrame(index=range(dataset_multi.index.size), 
                            columns=['h average'])

df_real['h average'] = real_concat
df_forcasted['h average'] = model_concat
df_train['h average'][0:len(trainPredict)] = model_concat[0:len(trainPredict)]
df_test['h average'][len(trainPredict):] = model_concat[len(trainPredict):]

#ricreo l'indice
index = pd.date_range(start = df.index[0],
                      periods = dataset_multi.index.size, freq='H')
df_forcasted.index = index
df_real.index = index
df_train.index = index 
df_test.index = index

# Let's take a look at the predictions

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))

#ax.plot(range(train_y[:,0].size) , train_y[:,0], linewidth=1.5, label = 'training')   
#ax.plot(range(train_y[:,0].size, scaled[:,0].size), test_y[:,0], linewidth=1.5, label = 'testing') 

ax.plot(df_train, linewidth=1.5, label = 'trainPredict', alpha=0.6)   
ax.plot(df_test, linewidth=1.5, label = 'testPredict', alpha=0.6) 
ax.plot(df_real, linewidth=1.5, label = 'real', alpha=0.3, color='k')
ax.set_ylabel('(L/s)', fontsize=22)
ax.grid(True)       
ax.legend(fontsize=18)
fig.autofmt_xdate()

In [ ]:
start_date = datetime.datetime.strptime(('2013-09-01'), '%Y-%m-%d')
end_date = start_date + datetime.timedelta(days = 10)
fig, (ax) = plt.subplots(1, figsize=(15, 7))
ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
ax.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H'))
ax.format_xdata = mdates.DateFormatter('%Y-%m')
ax.plot(df_real[start_date:end_date], label = 'True')
ax.plot(df_train[start_date:end_date])
ax.plot(df_test[start_date:end_date], label = 'prediction')
ax.set_ylabel('(L/s)')
ax.set_title('Hourly Forecasting')
ax.legend()
# ax.legend(('real data','training','validation'))
ax.grid(True)  